参考: https://acro-engineer.hatenablog.com/entry/2020/12/15/120000

In [ ]:
from functools import partial

import pandas as pd
from sklearn.model_selection import KFold
import xfeat
import lightgbm as lgb
import optuna

In [ ]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [ ]:
# 学習データとテストデータをまとめる
df_all = pd.concat([df_train, df_test], axis=0)
df_all

In [ ]:
# カテゴリデータを抽出する
xfeat.SelectCategorical().fit_transform(df_all)

In [ ]:
# 数値データを抽出する
xfeat.SelectNumerical().fit_transform(df_all)

In [ ]:
# 数値の型をカテゴリに変換すると、カテゴリ変数として認識される
df_all['Pclass'] = df_all['Pclass'].astype('category')
xfeat.SelectCategorical().fit_transform(df_all).columns

In [ ]:
# ラベルエンコーディング
# suffixに何か指定すると、元のデータも残る
encoder = xfeat.Pipeline([
	xfeat.SelectCategorical(exclude_cols=['Name', 'Ticket']),
	xfeat.LabelEncoder(output_suffix='_le'),
])

df_le = encoder.fit_transform(df_all)
df_le

In [ ]:
# ラベルエンコーディング
# 同じエンコーダを使ってもデータが違えば同じ値でも違う値にエンコードされる
df_le = encoder.fit_transform(df_test)		# 上の例では全体データを用いた
df_le

In [ ]:
# 重要度に基づく特徴量選択用にデータを作成する
df_train_le = pd.merge(df_train.drop(['Sex', 'Cabin', 'Embarked', 'Name', 'Ticket'], axis=1), df_le.drop(['Sex', 'Cabin', 'Embarked'], axis=1), how='right', left_index=True, right_index=True)
df_train_le

In [ ]:
# Target Encoding
# Target Encodingの説明はhttps://www.codexa.net/target_encoding/が参考になる
fold = KFold(n_splits=5, shuffle=False)
encoder = xfeat.TargetEncoder(
	input_cols=['Cabin'],
	target_col='Survived',
	fold=fold,
	output_suffix='_te',
)

df_te = encoder.fit_transform(df_train)			# 学習データでやること
df_te

In [ ]:
# 重要度に基づく特徴量選択用にデータを作成する
df_train_le = pd.merge(df_train_le, df_te[['Cabin_te']], how='left', left_index=True, right_index=True)
df_train_le

In [ ]:
# 複数のカテゴリ変数を組み合わせる
encoder = xfeat.Pipeline([
	xfeat.SelectCategorical(exclude_cols=['Name', 'Ticket']),
	xfeat.ConcatCombination(output_suffix='_cc', r=2),
])

# category型のままだとエラーになるのでstr型に変換する
encoder.fit_transform(xfeat.SelectCategorical().fit_transform(df_all).astype(str))

In [ ]:
# 数値データの加算をする
encoder = xfeat.Pipeline([
	xfeat.SelectNumerical(),
	xfeat.ArithmeticCombinations(
		input_cols=['SibSp', 'Parch'],
		output_suffix='_sum',
		operator='+',
		r=2
		),
])

encoder.fit_transform(df_all)

In [ ]:
# 数値データの減算をする
encoder = xfeat.Pipeline([
	xfeat.SelectNumerical(),
	xfeat.ArithmeticCombinations(
		input_cols=['SibSp', 'Parch'],		# 前から後を引く
		output_suffix='_diff',
		operator='-',
		r=2
		),
])

encoder.fit_transform(df_all)

In [ ]:
# 軸別の集計をし、その属性に当てはまるレコードの列に追加する
# df: 元のデータに列が追加されたもの
# cols: 新しく追加された列の名前
df, cols = xfeat.aggregation(
	df_all,
	group_key='Sex',		# 複数を軸にすることはできない
	group_values=['Age', 'Fare'],
	agg_methods=['mean', 'max', 'min', 'std']
)

In [ ]:
df

In [ ]:
cols

In [ ]:
# 集計値確認
df_all.groupby('Sex').describe()

In [ ]:
# LightGBMの特徴量重要度に合わせた選択
lgbm_params = {
	'objective': 'binary',
	'metric': 'binary_error'
}
fit_kwargs = {
	'num_boost_round': 10
}

selector = xfeat.GBDTFeatureSelector(
	target_col='Survived',
	threshold=0.5,				# 選択される変数の割合
	lgbm_params=lgbm_params,
	lgbm_fit_kwargs=fit_kwargs,
)
selector.fit_transform(df_train_le)

In [ ]:
# optunaと組み合わせる
lgbm_params = {
	'objective': 'binary',
	'metric': 'binary_error',
	'verbosity': -1,
}

def objective(df, selector, trial):
    selector.set_trial(trial)
    selector.fit(df)
    input_cols = selector.get_selected_cols()

    # Hyper Parameter Tuning するパラメータと範囲を設定する
    lgbm_params = {
        'num_leaves': trial.suggest_int("num_leaves", 3, 10),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
    }
    lgbm_params.update(lgbm_params)

    # Evaluate with selected columns
    train_set = lgb.Dataset(df[input_cols], label=df["Survived"])
    scores = lgb.cv(lgbm_params, train_set, num_boost_round=100, stratified=False, seed=1)

    binary_error_score = scores['l2-mean'][-1]
    return 1 - binary_error_score


# 特徴量探索のための説明変数を設定する
input_cols = list(df_train_le.columns)
input_cols.remove('Survived')

# 特徴量探索器を作成する
selector = xfeat.GBDTFeatureExplorer(
    input_cols=input_cols,
    target_col="Survived",
    fit_once=True,
    threshold_range=(0.8, 1.0),
    lgbm_params=lgbm_params,
)

# Hyper Parameter Tuning を行う
study = optuna.create_study(direction="minimize")
study.optimize(partial(objective, df_train_le, selector), n_trials=100)

# 選択された特徴量を確認する
selector.from_trial(study.best_trial)
print("Selected columns:", selector.get_selected_cols())

In [ ]:
study.best_params

In [ ]:
study.best_value